Script for Reddit - Credentials in passcode file - using PRAWC - change password_file_path to your systems

In [ ]:
import os
import requests
import praw

# Set the path to the password file
password_file_path = r"C:\Users\USERNAME\OneDrive\Documents\GitHub\non_share\passcode.txt"

# Open the password file and read its contents
with open(password_file_path, 'r') as f:
    contents = f.read()

# Split the contents into lines
lines = contents.split('\n')

# Initialize variables to store the client ID and secret key
CLIENT_ID = None
SECRET_KEY = None
pw_reddit = None

# Loop through the lines to find the client ID and secret key
for line in lines:
    if line.startswith('CLIENT_ID'):
        CLIENT_ID = line.split('=')[1].strip()
    elif line.startswith('SECRET_KEY'):
        SECRET_KEY = line.split('=')[1].strip()
    elif line.startswith('pw_reddit'):
        pw_reddit = line.split('=')[1].strip()
    elif line.startswith('reddit_user'):
        reddit_user = line.split('=')[1].strip()

# Print the client ID and secret key
print(f'CLIENT_ID: {CLIENT_ID}')
print(f'SECRET_KEY: {SECRET_KEY}')
print(f'pw_reddit: {pw_reddit}')
print(f'reddit_user: {reddit_user}')


Define User_Agent - Follow Reddit PRAW Guidelines for API Access

In [ ]:
reddit = praw.Reddit(
    client_id = CLIENT_ID,
    client_secret = SECRET_KEY,
    #redirect_uri = 'https://localhost:8080',
    username = reddit_user,
    user_agent = 'ScriptCounterAPI/0.0.1 by ****',
)

# Authenticate the Reddit instance by accessing your user account
try:
    reddit.user.me()
except Exception as e:
    print(f"Error: {e}")

# Print the access token for your account
print(reddit.auth.scopes())


Test API - Access - change subreddit name your needs. top limit will retrieve the first 10 Top submissions in the subreddit.

In [ ]:
for submission in reddit.subreddit('dataisbeautiful').top(limit=10):
    print(submission.title)

If needed change access to read_only mode

In [ ]:
#reddit.read_only = True

In [ ]:
subreddit = reddit.subreddit('dataisbeautiful')
hot_dataib = subreddit.hot(limit=10)
for submission in hot_dataib:
    print(submission.title)

Import submissions in to Panda dataframe and use exclusion to remove

In [ ]:
import pandas as pd

# Get subreddit
subreddit = reddit.subreddit('dataisbeautiful')

# List of excluded words
exclusions = ['is', 'not', 'and', 'or', 'oc', '[oc]','how','many','you','need','to','of','if','by','the','i','for','a','-','an','with','on','—','_','as']

# Dictionary to store word counts
word_counts = {}

# Iterate through all posts in subreddit
for post in subreddit.new(limit=1000):
    # Split post title into words and convert to lowercase
    words = post.title.lower().split()
    # Iterate through words and update word counts
    for word in words:
        if word not in exclusions:
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1

# Create DataFrame from word_counts dictionary
df = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Count'])

# Write DataFrame to Excel file
df.to_excel('word_counts.xlsx', index=False)

In [ ]:
subreddit_name = 'dataisbeautiful'
subreddit = reddit.subreddit(subreddit_name)

top_submissions = subreddit.top(limit=10)  # get the top 10 submissions in the subreddit

for submission in top_submissions:
    print(submission.title)

Using Exclusion List on retrieved submissions.

In [ ]:
import pandas as pd

# Get subreddit
subreddit = reddit.subreddit('dataisbeautiful')

# List of excluded words
exclusions = ['is', 'not', 'and', 'or', 'oc', '[oc]','how','many','you','need','to','of','if','by','the','i','for','a','-','an','with','on','—','_','as','in','from','are','at','what','visualized','visualization','&']

# Dictionary to store word counts
word_counts = {}

# Counter variable
count = 0

# Loop through subreddit
for post in subreddit.new(limit=None):
    # Split post title into words and convert to lowercase
    words = post.title.lower().split()
    # Iterate through words and update word counts
    for word in words:
        if word not in exclusions:
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1
    # Increment counter
    count += 1
    # Break loop after two rounds
    if count >= 3000:
        break

# Create DataFrame from word_counts dictionary
df = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Count'])

# Write DataFrame to Excel file
df.to_excel('word_counts.xlsx', index=False)

Exporting to Excel in Batches - using regex to clean the title from special characters

In [ ]:
import pandas as pd
import praw
import re

subreddit_name = 'dataisbeautiful'
subreddit = reddit.subreddit(subreddit_name)

post_titles = []
post_dates = []
for post in subreddit.new(limit=3000):
    title = post.title.lower()  # convert to lowercase
    title = re.sub(r'[^a-zA-Z0-9\s]', '', title)  # remove special characters
    post_titles.append(title)
    post_dates.append(post.created_utc)

df = pd.DataFrame({'text': post_titles, 'Date': post_dates})
df['Date'] = pd.to_datetime(df['Date'], unit='s')  # Convert UNIX timestamps to datetime objects

file_exists = False
try:
    with pd.ExcelFile('reddit_posts.xlsx') as xls:
        file_exists = True
except FileNotFoundError:
    pass

with pd.ExcelWriter('reddit_posts.xlsx', engine='openpyxl', mode='a' if file_exists else 'w') as writer:
    sheet_name = 'dataisbeautiful'
    df.to_excel(writer, index=False, header=not file_exists, sheet_name=sheet_name, if_sheet_exists="replace")

print(f"Exported {len(df)} posts to reddit_posts.xlsx")


In [ ]:
display(df)

Using Pushshift API to retrieve Reddit posts in batches for i in range(0, 5000 endpoint, 100 submissions per batch). Even if the final export to excel should fail - the submissions should be in the pandas dataframe - you can check the dataframe with the code below. If the export to Excel should fail try to export it to CSV.

In [ ]:
import time
import pandas as pd
from pmaw import PushshiftAPI
import signal

# Set the subreddit name and endpoint URL
subreddit_name = 'dataisbeautiful'

# Initialize Pushshift API
api = PushshiftAPI()

# Create an empty dataframe to store the posts
df = pd.DataFrame()

# Define a signal handler to gracefully exit the script
def signal_handler(signal, frame):
    print("\nKeyboardInterrupt detected. Exiting script.")
    df.to_excel('reddit_posts.xlsx', index=False)
    exit(0)

# Set the signal handler for SIGINT
signal.signal(signal.SIGINT, signal_handler)

# Loop through the subreddit in batches of 100 with a delay of 1 second in between
post_list = []
max_attempts = 3
attempt_count = 0
for i in range(0, 5000, 100):
    # Construct the API request
    request = {
        "subreddit": subreddit_name,
        "size": 100,
        "after": f"{i}d"
    }

    # Send the API request
    attempts = 0
    while attempts < max_attempts:
        try:
            data = api.search_submissions(**request)
            break
        except Exception as e:
            print(f"Error: {e}. Waiting for 60 seconds before retrying...")
            time.sleep(60)
            attempts += 1
    else:
        print("Reached maximum number of attempts. Skipping batch.")
        continue

    # Append the posts to the list
    for post in data:
        post_list.append({
            'id': post['id'],
            'title': post['title'],
            'author': post['author'] if 'author' in post else None,
            'created_utc': post['created_utc'],
            'score': post['score'],
            'num_comments': post['num_comments'],
            'permalink': f"https://www.reddit.com{post['permalink']}",
            'url': post['url']
        })

    # Print status message
    print(f"Batch added - current item number: {i+100}")

    # Concatenate the list to the dataframe every 100 posts
    if len(post_list) >= 100:
        df = pd.concat([df, pd.DataFrame(post_list)], ignore_index=True)
        post_list = []
        time.sleep(1)

# Concatenate remaining posts to dataframe
if post_list:
    df = pd.concat([df, pd.DataFrame(post_list)], ignore_index=True)

# Export DataFrame to Excel file
df.to_excel('reddit_posts.xlsx', index=False)

print(f"Exported {len(df)} posts to reddit_posts.xlsx")

Display pandas dataframe

In [ ]:
display(df)

In [ ]:
# Export DataFrame to Excel file
df.to_excel('batch_reddit_posts.xlsx', index=False)

print(f"Exported {len(df)} posts to 5000 batch_reddit_posts.xlsx")

In [ ]:
# save the DataFrame to a CSV file
df.to_csv('my_data.csv', index=False)